<a href="https://colab.research.google.com/github/YangJeongA/class2022Spring/blob/main/220530_tensorflow_hub%2C_pytorch_hub_for_more_AI_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hugging face외의 다른 AI model들 살펴보기

facebook에서 제공하는 pytorch, google에서 제공하는 tensorflow에 대해 알아볼 것

- Tensorflow와 pytorch는 둘 다 python 내의 libraries(package)
  - AI와 관련된 함수들!
  - 어떤 알고리즘을 두 가지 모두로 짤 수 있지만 각자의 장점이 있음
  - huggingface에 있는 AI model들도 대부분 pytorch아니면 tensorflow로 만들어진 것들

# [Tensorflow hub](https://www.tensorflow.org/hub)
- fine-tuning: (matrix) pre-trained된 모델이 있을 때, 다른 새로운 데이터를 가져와서 model을 확장(변경)시키는 것 

image classification (e.g. [inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5))

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

In [ ]:
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")


# hub에서 모델을 가져옴

In [ ]:
from PIL import Image
import requests
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
import numpy as np
image = np.array(image)
# reshape into shape [batch_size, height, width, num_channels]
image = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])       #reshape해서 3차원의 image를 4차원으로 만들어줌 -> model 자체가 4차원이라서

# Use `convert_image_dtype` to convert to floats in the [0,1] range.  (0~255의 실수에서 [0,1]의 range로 바꿔줌->model이 그렇게 훈련되어 있기 때문)
image = tf.image.convert_image_dtype(image, tf.float32)

# image는 RGB의 형태이므로 3차원 형태
# image를 4차원으로 np.array화 해주어야 데이터로 사용 가능 (model특성상)

In [ ]:
logits = model(image)
np.argmax(logits)    # np.argmax -> 가장 큰 값을 출력해 줌 -> 이 값에 해당하는 순서의 이미지가 가장 확률이 높은 이미지인 것

# 이 단계에서 model을 사용 -> model안에 행렬들이 들어있고 image를 데이터로 넣는 것


> https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt



#[Pythorch Hub](https://pytorch.org/hub/research-models)

image generation (e.g. [PGAN](https://pytorch.org/hub/facebookresearch_pytorch-gan-zoo_pgan/))

In [ ]:
import torch
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-512', pretrained=True)

In [ ]:
num_images = 4
noise, _ = model.buildNoiseData(num_images)
with torch.no_grad():
    generated_images = model.test(noise)
# noise가 random한 image(얼굴)를 만들어줌 

# let's plot these images using torchvision and matplotlib
import matplotlib.pyplot as plt
import torchvision
grid = torchvision.utils.make_grid(generated_images.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())